In [1]:
#!/usr/bin/env python
import sys
import pandas as pd
import pymongo
import json
import os
import numpy


In [3]:
file = 'globalterrorismdb_0718dist.csv'
terror_df = pd.read_csv(file, encoding = "ISO-8859-1")
low_memory=False,

terror_df.head()

C:\Users\Christopher Sloat\anaconda\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [4]:
column_names =list(terror_df)
column_names

['eventid',
 'iyear',
 'imonth',
 'iday',
 'approxdate',
 'extended',
 'resolution',
 'country',
 'country_txt',
 'region',
 'region_txt',
 'provstate',
 'city',
 'latitude',
 'longitude',
 'specificity',
 'vicinity',
 'location',
 'summary',
 'crit1',
 'crit2',
 'crit3',
 'doubtterr',
 'alternative',
 'alternative_txt',
 'multiple',
 'success',
 'suicide',
 'attacktype1',
 'attacktype1_txt',
 'attacktype2',
 'attacktype2_txt',
 'attacktype3',
 'attacktype3_txt',
 'targtype1',
 'targtype1_txt',
 'targsubtype1',
 'targsubtype1_txt',
 'corp1',
 'target1',
 'natlty1',
 'natlty1_txt',
 'targtype2',
 'targtype2_txt',
 'targsubtype2',
 'targsubtype2_txt',
 'corp2',
 'target2',
 'natlty2',
 'natlty2_txt',
 'targtype3',
 'targtype3_txt',
 'targsubtype3',
 'targsubtype3_txt',
 'corp3',
 'target3',
 'natlty3',
 'natlty3_txt',
 'gname',
 'gsubname',
 'gname2',
 'gsubname2',
 'gname3',
 'gsubname3',
 'motive',
 'guncertain1',
 'guncertain2',
 'guncertain3',
 'individual',
 'nperps',
 'nperpcap',
 

In [5]:
dashboard_df = terror_df.loc[:, ["nkill","nwound","success","region","country_txt","iyear","region_txt",]]
dashboard_df.head()

,nkill,nwound,success,region,country_txt,iyear,region_txt
0,1.0,0.0,1,2,Dominican Republic,1970,Central America & Caribbean
1,0.0,0.0,1,1,Mexico,1970,North America
2,1.0,0.0,1,5,Philippines,1970,Southeast Asia
3,NaN,NaN,1,8,Greece,1970,Western Europe
4,NaN,NaN,1,4,Japan,1970,East Asia


In [6]:
total_attacks = dashboard_df["success"].count()
total_attacks
total_wounded = dashboard_df["nwound"].count()
total_wounded

165380

In [7]:
total_fatalities = dashboard_df["nkill"].sum()
total_fatalities

411868.0

In [8]:
dashboard_df["total_fatalities"] = total_fatalities
dashboard_df["total_attacks"] = total_attacks
dashboard_df["total_wounded"] = total_wounded
dashboard_df.head()

,nkill,nwound,success,region,country_txt,iyear,region_txt,total_fatalities,total_attacks,total_wounded
0,1.0,0.0,1,2,Dominican Republic,1970,Central America & Caribbean,411868.0,181691,165380
1,0.0,0.0,1,1,Mexico,1970,North America,411868.0,181691,165380
2,1.0,0.0,1,5,Philippines,1970,Southeast Asia,411868.0,181691,165380
3,NaN,NaN,1,8,Greece,1970,Western Europe,411868.0,181691,165380
4,NaN,NaN,1,4,Japan,1970,East Asia,411868.0,181691,165380


In [9]:
dashboard_df["region_txt"].unique()

array(['Central America & Caribbean', 'North America', 'Southeast Asia',
       'Western Europe', 'East Asia', 'South America', 'Eastern Europe',
       'Sub-Saharan Africa', 'Middle East & North Africa',
       'Australasia & Oceania', 'South Asia', 'Central Asia'],
      dtype=object)

In [10]:
success_failure = dashboard_df["success"].value_counts(1)

success_failure.head()

1    0.889598
0    0.110402
Name: success, dtype: float64

In [11]:
dashboard_df["tsuccess"] = .889598
dashboard_df["tfailure"] = .110402
dashboard_df.head()

,nkill,nwound,success,region,country_txt,iyear,region_txt,total_fatalities,total_attacks,total_wounded,tsuccess,tfailure
0,1.0,0.0,1,2,Dominican Republic,1970,Central America & Caribbean,411868.0,181691,165380,0.889598,0.110402
1,0.0,0.0,1,1,Mexico,1970,North America,411868.0,181691,165380,0.889598,0.110402
2,1.0,0.0,1,5,Philippines,1970,Southeast Asia,411868.0,181691,165380,0.889598,0.110402
3,NaN,NaN,1,8,Greece,1970,Western Europe,411868.0,181691,165380,0.889598,0.110402
4,NaN,NaN,1,4,Japan,1970,East Asia,411868.0,181691,165380,0.889598,0.110402


In [12]:
region_values = dashboard_df["region_txt"].value_counts()

In [13]:
print(region_values)

Middle East & North Africa     50474
South Asia                     44974
South America                  18978
Sub-Saharan Africa             17550
Western Europe                 16639
Southeast Asia                 12485
Central America & Caribbean    10344
Eastern Europe                  5144
North America                   3456
East Asia                        802
Central Asia                     563
Australasia & Oceania            282
Name: region_txt, dtype: int64


In [14]:
region_sum_df = pd.DataFrame(region_values)

In [15]:
region_sum_df.rename(columns={"region_txt":"total_terror"}, inplace=True)


In [16]:

#region_sum_df.to_json(r'region_sum')
#dashboard_df.to_json(r'dashboard_df')

In [17]:
incident_pyear = dashboard_df['iyear'].value_counts()

In [18]:
incident_pyear_df = pd.DataFrame(incident_pyear)

incident_pyear_df.head()

,iyear
2014,16903
2015,14965
2016,13587
2013,12036
2017,10900


In [19]:
incident_pyear_df.rename(columns={"iyear":"incidents"}, inplace=True)


In [20]:
#incident_pyear_df.to_json(r'incidents_per_year')
incident_pyear_df.head()

,incidents
2014,16903
2015,14965
2016,13587
2013,12036
2017,10900


In [21]:
country_values = dashboard_df["country_txt"].value_counts()
country_values.mean()
#bin countries based on 3 colors

886.2975609756097

In [22]:
country_bins = pd.DataFrame(country_values)
country_bins.rename(columns={"country_txt":"incidents"}, inplace=True)
country_bins.head()

,incidents
Iraq,24636
Pakistan,14368
Afghanistan,12731
India,11960
Colombia,8306


In [23]:
dashboard_df.head()

,nkill,nwound,success,region,country_txt,iyear,region_txt,total_fatalities,total_attacks,total_wounded,tsuccess,tfailure
0,1.0,0.0,1,2,Dominican Republic,1970,Central America & Caribbean,411868.0,181691,165380,0.889598,0.110402
1,0.0,0.0,1,1,Mexico,1970,North America,411868.0,181691,165380,0.889598,0.110402
2,1.0,0.0,1,5,Philippines,1970,Southeast Asia,411868.0,181691,165380,0.889598,0.110402
3,NaN,NaN,1,8,Greece,1970,Western Europe,411868.0,181691,165380,0.889598,0.110402
4,NaN,NaN,1,4,Japan,1970,East Asia,411868.0,181691,165380,0.889598,0.110402


In [24]:
bins = [0,22,190, 24636]
group_names = ["Green", "Yellow", "Red"]

In [25]:
country_bins["danger_color"] = pd.cut(country_bins["incidents"], bins, labels=group_names)
country_bins.head()

,incidents,danger_color
Iraq,24636,Red
Pakistan,14368,Red
Afghanistan,12731,Red
India,11960,Red
Colombia,8306,Red


In [26]:
country_color = country_bins.groupby("danger_color")
country_color.head()

,incidents,danger_color
Iraq,24636,Red
Pakistan,14368,Red
Afghanistan,12731,Red
India,11960,Red
Colombia,8306,Red
Ethiopia,190,Yellow
Tajikistan,188,Yellow
Bosnia-Herzegovina,159,Yellow
Rwanda,159,Yellow
Belgium,154,Yellow


In [27]:
country_danger_color = pd.DataFrame(country_bins)
country_danger_color.head()

,incidents,danger_color
Iraq,24636,Red
Pakistan,14368,Red
Afghanistan,12731,Red
India,11960,Red
Colombia,8306,Red


In [ ]:
# HERE IS A LIST OF ALL THE CLEANED AND READY DATA FRAMES
# dashboard_df = Most of the data for the dashboards. It is missing the bin information and the region totals of incidents
# country_danger_color = The bined amount of incidents per country and labeled green, yellow and red
# incidents_per_year = The amount of incidents that happened globally per year
# region_sum = Amount of incidents that has happened per region over all years
# total fatalities, total wounded, total attacks, success percent, and failure percent are all in the dashboard_df but I will list the values below
# total fatalities = 411868.0
# total attacks = 181691
# total wounded = 165380
# success percentage = 0.889598 or 89$
# failure percent = 0.110402 or 11%
#dashboard_df
#Country_danger_color
#incidents_per_year
#region_sum